In [3]:
import pandas as pd
import numpy as np # linear algebra

# Load all sheets
freight = pd.read_excel('/Users/surya/Downloads/Supply chain logisitcs problem.xlsx', sheet_name='FreightRates')
order = pd.read_excel('/Users/surya/Downloads/Supply chain logisitcs problem.xlsx', sheet_name='OrderList')
plant_ports = pd.read_excel('/Users/surya/Downloads/Supply chain logisitcs problem.xlsx', sheet_name='PlantPorts')
prod_per_plant = pd.read_excel('/Users/surya/Downloads/Supply chain logisitcs problem.xlsx', sheet_name='ProductsPerPlant')
vmi_cust = pd.read_excel('/Users/surya/Downloads/Supply chain logisitcs problem.xlsx', sheet_name='VmiCustomers')
wh_cap = pd.read_excel('/Users/surya/Downloads/Supply chain logisitcs problem.xlsx', sheet_name='WhCapacities')
wh_costs = pd.read_excel('/Users/surya/Downloads/Supply chain logisitcs problem.xlsx', sheet_name='WhCosts')

# Display the heads of all DataFrames
print("Freight Rates:")
print(freight.head(), "\n")

print("Order List:")
print(order.head(), "\n")

print("Plant Ports:")
print(plant_ports.head(), "\n")

print("Products Per Plant:")
print(prod_per_plant.head(), "\n")

print("VMI Customers:")
print(vmi_cust.head(), "\n")

print("Warehouse Capacities:")
print(wh_cap.head(), "\n")

print("Warehouse Costs:")
print(wh_costs.head(), "\n")


Freight Rates:
  Carrier orig_port_cd dest_port_cd  minm_wgh_qty  max_wgh_qty svc_cd  \
0  V444_6       PORT08       PORT09         250.0       499.99    DTD   
1  V444_6       PORT08       PORT09          65.0        69.99    DTD   
2  V444_6       PORT08       PORT09          60.0        64.99    DTD   
3  V444_6       PORT08       PORT09          50.0        54.99    DTD   
4  V444_6       PORT08       PORT09          35.0        39.99    DTD   

   minimum cost    rate mode_dsc  tpt_day_cnt Carrier type  
0       43.2272  0.7132   AIR               2  V88888888_0  
1       43.2272  0.7512   AIR               2  V88888888_0  
2       43.2272  0.7892   AIR               2  V88888888_0  
3       43.2272  0.8272   AIR               2  V88888888_0  
4       43.2272  1.0552   AIR               2  V88888888_0   

Order List:
       Order ID Order Date Origin Port Carrier  TPT Service Level  \
0  1.447296e+09 2013-05-26      PORT09   V44_3    1           CRF   
1  1.447158e+09 2013-05-26  

In [2]:
!pip install openpyxl

In [4]:
dict(zip(freight['orig_port_cd'],freight['dest_port_cd']))


{'PORT08': 'PORT09',
 'PORT10': 'PORT09',
 'PORT09': 'PORT09',
 'PORT11': 'PORT09',
 'PORT04': 'PORT09',
 'PORT02': 'PORT09',
 'PORT03': 'PORT09',
 'PORT07': 'PORT09',
 'PORT05': 'PORT09',
 'PORT06': 'PORT09'}

In [5]:
#check null values 
freight.isnull().sum()


Carrier         0
orig_port_cd    0
dest_port_cd    0
minm_wgh_qty    0
max_wgh_qty     0
svc_cd          0
minimum cost    0
rate            0
mode_dsc        0
tpt_day_cnt     0
Carrier type    0
dtype: int64

In [6]:
order.isnull().sum()


Order ID                0
Order Date              0
Origin Port             0
Carrier                 0
TPT                     0
Service Level           0
Ship ahead day count    0
Ship Late Day count     0
Customer                0
Product ID              0
Plant Code              0
Destination Port        0
Unit quantity           0
Weight                  0
dtype: int64

In [7]:
plant_ports.isnull().sum()


Plant Code    0
Port          0
dtype: int64

In [8]:
prod_per_plant.isnull().sum()


Plant Code    0
Product ID    0
dtype: int64

In [9]:
vmi_cust.isnull().sum()


Plant Code    0
Customers     0
dtype: int64

In [10]:
wh_cap.isnull().sum()


Plant ID           0
Daily Capacity     0
dtype: int64

In [11]:
wh_costs.isnull().sum()


WH           0
Cost/unit    0
dtype: int64

In [12]:
# Select relevant columns from the 'order' DataFrame for analysis
order = order[['Order Date', 'Order ID', 'Carrier', 'Origin Port', 'Destination Port', 'Unit quantity', 'Weight']]

# Merge the 'order' DataFrame with the 'plant_ports' DataFrame to associate each order's origin port with its corresponding plant code
df_merge = pd.merge(order, plant_ports, left_on=['Origin Port'], right_on=['Port'])

# Rename the column 'Plant Code' to 'origin_plant_code' for better clarity
df_merge = df_merge.rename(columns={'Plant Code': 'origin_plant_code'})

# Drop the redundant 'Port' column after the first merge
df_merge = df_merge.drop(['Port'], axis=1)

# Merge the updated DataFrame with the 'plant_ports' DataFrame again to associate each order's destination port with its corresponding plant code
df_merge = pd.merge(df_merge, plant_ports, left_on=['Destination Port'], right_on=['Port'])

# Drop the redundant 'Port' column after the second merge
df_merge = df_merge.drop(['Port'], axis=1)

# Rename the column 'Plant Code' to 'destination_plant_code' for better clarity
df_merge = df_merge.rename(columns={'Plant Code': 'destination_plant_code'})

# Display the first few rows of the final merged DataFrame
df_merge.head()


,Order Date,Order ID,Carrier,Origin Port,Destination Port,Unit quantity,Weight,origin_plant_code,destination_plant_code
0,2013-05-26,1.447296e+09,V44_3,PORT09,PORT09,808,14.30,PLANT16,PLANT16
1,2013-05-26,1.447158e+09,V44_3,PORT09,PORT09,3188,87.94,PLANT16,PLANT16
2,2013-05-26,1.447139e+09,V44_3,PORT09,PORT09,2331,61.20,PLANT16,PLANT16
3,2013-05-26,1.447364e+09,V44_3,PORT09,PORT09,847,16.16,PLANT16,PLANT16
4,2013-05-26,1.447364e+09,V44_3,PORT09,PORT09,2163,52.34,PLANT16,PLANT16


In [13]:
# Group the 'freight' DataFrame by 'Carrier', 'orig_port_cd', and 'dest_port_cd' to calculate aggregated metrics
freight_new = freight.groupby(['Carrier', 'orig_port_cd', 'dest_port_cd']).agg({
    'minm_wgh_qty': lambda x: sum(x) / len(x),  # Calculate the average for 'minm_wgh_qty'
    'max_wgh_qty': lambda x: sum(x) / len(x),  # Calculate the average for 'max_wgh_qty'
    'rate': lambda x: sum(x) / len(x),         # Calculate the average for 'rate'
    'minimum cost': lambda x: sum(x) / len(x) # Calculate the average for 'minimum cost'
}).reset_index()  # Reset the index to return a flat DataFrame

# Rename columns for clarity and standardization
freight_new = freight_new.rename(columns={
    'minimum cost': 'min_cost', 
    'minm_wgh_qty': 'min_wgh_qty'
})

# Display the resulting DataFrame with aggregated metrics
freight_new


,Carrier,orig_port_cd,dest_port_cd,min_wgh_qty,max_wgh_qty,rate,min_cost
0,V444_0,PORT02,PORT09,570.000000,20569.990000,0.084480,23.838400
1,V444_0,PORT03,PORT09,0.000000,5000.000000,12.694200,28.944200
2,V444_0,PORT04,PORT09,570.000000,20569.990000,0.052000,2.477200
3,V444_0,PORT09,PORT09,0.000000,5000.000000,12.778400,31.278400
4,V444_1,PORT02,PORT09,197.556000,1190.745000,0.291140,23.557340
5,V444_1,PORT04,PORT09,197.556000,1190.745000,0.242000,20.909780
6,V444_1,PORT05,PORT09,170.130000,1001.952500,0.457895,35.954091
7,V444_1,PORT06,PORT09,45.473493,111.272697,2.492247,6.917300
8,V444_1,PORT10,PORT09,78.945000,12570.433750,0.431400,12.350850
9,V444_2,PORT02,PORT09,570.000000,20569.990000,0.079040,9.027200


In [15]:
#Generate Cost Between Origin and Destination

#From PORT02 to PORT11 in chronological order w/ the same destination to PORT09
cost = freight_new.groupby(['orig_port_cd','dest_port_cd'])['min_cost'].min().unstack()
cost = round(cost,0)
cost = cost[cost.index.isin(df_merge['Origin Port'])]
cost_pulp = cost.to_numpy().tolist()
cost_pulp

[[2.0], [8.0], [24.0]]

In [16]:
cost


dest_port_cd,PORT09
orig_port_cd,
PORT04,2.0
PORT05,8.0
PORT09,24.0


In [17]:
wh_cap


,Plant ID,Daily Capacity
0,PLANT15,11
1,PLANT17,8
2,PLANT18,111
3,PLANT05,385
4,PLANT02,138
5,PLANT01,1070
6,PLANT06,49
7,PLANT10,118
8,PLANT07,265
9,PLANT14,549


In [18]:
# Get unique and sorted lists of origin ports and destination ports from the merged DataFrame
origins = df_merge['Origin Port'].unique().tolist()
origins = sorted(origins)
destinations = df_merge['Destination Port'].unique().tolist()

# Aggregate data for each origin port
df_origin = df_merge.groupby(['Origin Port']).agg({
    'Order ID': 'nunique',       # Count the unique number of orders
    'Order Date': 'nunique',     # Count the unique order dates
    'Unit quantity': 'sum',      # Sum of unit quantities
    'Weight': 'sum'              # Sum of weights
}).reset_index()

# Calculate average unit quantity and average weight per order date for each origin port
df_origin['avg_qty'] = round(df_origin['Unit quantity'] / df_origin['Order Date'], 0)
df_origin['avg_weight'] = round(df_origin['Weight'] / df_origin['Order Date'], 0)

# Drop columns that are no longer needed in the origin summary DataFrame
df_origin = df_origin.drop(['Order ID', 'Unit quantity', 'Weight'], axis=1)

# Aggregate data for each destination port
df_dest = df_merge.groupby(['Destination Port']).agg({
    'Order ID': 'nunique',       # Count the unique number of orders
    'Order Date': 'nunique',     # Count the unique order dates
    'Unit quantity': 'sum',      # Sum of unit quantities
    'Weight': 'sum'              # Sum of weights
}).reset_index()

# Calculate average unit quantity and average weight per order date for each destination port
df_dest['avg_qty'] = round(df_dest['Unit quantity'] / df_dest['Order Date'], 0)
df_dest['avg_weight'] = round(df_dest['Weight'] / df_dest['Order Date'], 0)

# Drop columns that are no longer needed in the destination summary DataFrame
df_dest = df_dest.drop(['Order ID', 'Unit quantity', 'Weight'], axis=1)

# Display the resulting DataFrame summarizing destination port data
df_dest


,Destination Port,Order Date,avg_qty,avg_weight
0,PORT09,1,205120319.0,1088803.0


In [19]:
df_origin


,Origin Port,Order Date,avg_qty,avg_weight
0,PORT04,1,204874838.0,1056632.0
1,PORT05,1,348.0,2.0
2,PORT09,1,245133.0,32168.0


In [20]:
supply = dict(zip(df_origin['Origin Port'],df_origin['avg_weight']))
demand = dict(zip(df_dest['Destination Port'], df_dest['avg_weight']))

#All Variables for Routing Optimization 
# supply['PORT04'] = 1013
# supply['PORT05'] = 385
# supply['PORT09'] = 457
# demand['PORT09'] = 457
print(f'All Origin Nodes : {origins}')
print(f'Origin Supply : {supply}')
print(f'All Destination Nodes : {destinations}')
print(f'Destination Demand : {demand}')
print(f'Costs From Origin to Destination : {cost_pulp}')

All Origin Nodes : ['PORT04', 'PORT05', 'PORT09']
Origin Supply : {'PORT04': 1056632.0, 'PORT05': 2.0, 'PORT09': 32168.0}
All Destination Nodes : ['PORT09']
Destination Demand : {'PORT09': 1088803.0}
Costs From Origin to Destination : [[2.0], [8.0], [24.0]]


In [21]:
from pulp import *

# Create a dictionary of costs for transportation between origins and destinations
# 'makeDict' maps the cost data (cost_pulp) to a dictionary with keys as origin-destination pairs
costs = makeDict([origins, destinations], cost_pulp, 0)

# Define the linear programming problem
# The objective is to minimize the transportation costs ("LpMinimize")
prob = LpProblem("Port Distribution Problem", LpMinimize)

# Generate a list of all possible routes as (origin, destination) pairs
routes = [(o, d) for o in origins for d in destinations]

# Define decision variables for the problem
# 'vars' represents the number of units transported from each origin to each destination
# These are non-negative integer variables
vars = LpVariable.dicts("route", (origins, destinations), 0, None, LpInteger)

# Define the objective function to minimize the total transportation cost
# This is the sum of the product of transportation costs and the units transported for all routes
prob += (
    lpSum([vars[o][d] * costs[o][d] for (o, d) in routes]),
    "Sum of Transporting Costs"
)

# Display the defined problem
prob


/Users/surya/anaconda3/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Port_Distribution_Problem:
MINIMIZE
2.0*route_PORT04_PORT09 + 8.0*route_PORT05_PORT09 + 24.0*route_PORT09_PORT09 + 0.0
VARIABLES
0 <= route_PORT04_PORT09 Integer
0 <= route_PORT05_PORT09 Integer
0 <= route_PORT09_PORT09 Integer

In [22]:
# Supply Max Constraints from each Origin Node
for i, o in enumerate(origins):
    prob += (
        lpSum([vars[o][d] for d in destinations]) <= supply[o],
        "Supply_Constraint_Origin_%s_%d" % (o, i),  # Unique constraint name
    )

# Demand Min Constraints for each Destination Node
for i, d in enumerate(destinations):
    prob += (
        lpSum([vars[o][d] for o in origins]) >= demand[d],
        "Demand_Constraint_Destination_%s_%d" % (d, i),  # Unique constraint name
    )


In [23]:
prob.solve()
for v in prob.variables() :
    print(f'{v} : {v.varValue}')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/surya/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j7/t594hs655zzf0dkp2sm786kr0000gn/T/c69babf76fc0453092eecd5a38fbc932-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/j7/t594hs655zzf0dkp2sm786kr0000gn/T/c69babf76fc0453092eecd5a38fbc932-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 25 RHS
At line 30 BOUNDS
At line 34 ENDATA
Problem MODEL has 4 rows, 3 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.02

route_PORT04_PORT09 : 1056632.0
route_PORT05_PORT09 : 2.0
route_PORT09_PORT09 : 32169.0


## Supply Chain Distribution Version 2

In [24]:
# Group the 'freight' DataFrame by 'orig_port_cd' and 'dest_port_cd' to calculate aggregated metrics
cost_2 = freight.groupby(['orig_port_cd', 'dest_port_cd']).agg({
    'minimum cost': lambda x: round((x.max() + x.min()) / 2, 0)  # Calculate the average of the max and min values, rounded to the nearest integer
}).unstack()  # Unstack the grouped data to create a matrix-like structure with origins as rows and destinations as columns

# Convert the 'minimum cost' matrix to a NumPy array and then to a list for compatibility with optimization frameworks
cost_pulp_2 = cost_2['minimum cost'].to_numpy().tolist()

# Output the final cost matrix in list format
cost_pulp_2


[[60.0],
 [222.0],
 [52.0],
 [107.0],
 [60.0],
 [16.0],
 [125.0],
 [24.0],
 [33.0],
 [18.0]]

In [25]:
# Set the 'Port' column as the index in the 'plant_ports' DataFrame
plant_ports_filtered = plant_ports.set_index('Port')

# Filter the 'plant_ports_filtered' DataFrame to retain only rows where the index (Port) exists in the 'cost_2' DataFrame's index
plant_ports_filtered = plant_ports_filtered[plant_ports_filtered.index.isin(cost_2.index)]

# Reset the index and sort the resulting DataFrame by the 'Port' column
pp_filtered = plant_ports_filtered.reset_index().sort_values(by='Port')

# Display the final filtered and sorted DataFrame
pp_filtered


,Port,Plant Code
0,PORT02,PLANT01
6,PORT02,PLANT07
9,PORT02,PLANT10
1,PORT03,PLANT02
7,PORT04,PLANT08
8,PORT04,PLANT09
10,PORT04,PLANT11
11,PORT04,PLANT12
12,PORT04,PLANT13
2,PORT04,PLANT03


In [26]:
# Merge the filtered plant ports DataFrame with the warehouse capacities DataFrame
# The merge is performed on the 'Plant Code' column from 'pp_filtered' and the 'Plant ID' column from 'wh_cap'
# The 'how' parameter is set to 'left' to retain all rows from 'pp_filtered'
pp_merged = pd.merge(pp_filtered, wh_cap, left_on=['Plant Code'], right_on=['Plant ID'], how='left')

# Drop the redundant 'Plant ID' column from the merged DataFrame as it's no longer needed
pp_merged = pp_merged.drop(['Plant ID'], axis=1)

# Group the merged DataFrame by 'Port' and calculate the sum of 'Daily Capacity' for each port
pp_sum = pp_merged.groupby(['Port'])['Daily Capacity '].sum().reset_index()

# Display the final grouped and aggregated DataFrame
pp_sum


,Port,Daily Capacity
0,PORT02,1453
1,PORT03,138
2,PORT04,2076
3,PORT05,554
4,PORT06,434
5,PORT07,549
6,PORT08,11
7,PORT09,457
8,PORT10,8
9,PORT11,111


In [27]:
# Calculate the percentage of total daily capacity contributed by a specific port ('PORT09')
# Filter the 'pp_sum' DataFrame to get the 'Daily Capacity' for 'PORT09'
# Divide it by the total daily capacity across all ports in 'pp_sum'
perc_needed = pp_sum[pp_sum['Port'] == 'PORT09']['Daily Capacity '] / sum(pp_sum['Daily Capacity '])

# Display the calculated percentage for 'PORT09'
perc_needed


7    0.078916
Name: Daily Capacity , dtype: float64

In [28]:
pp_merged.columns


Index(['Port', 'Plant Code', 'Daily Capacity '], dtype='object')

In [29]:
# Extract unique origin port codes from the 'freight' DataFrame and sort them
f_origin = freight[['orig_port_cd']].drop_duplicates().sort_values(by='orig_port_cd')

# Extract unique destination port codes from the 'freight' DataFrame and sort them
f_dest = freight[['dest_port_cd']].drop_duplicates().sort_values(by='dest_port_cd')

# Merge the unique origin ports DataFrame ('f_origin') with the 'pp_sum' DataFrame
# This adds the corresponding 'Daily Capacity' for each origin port
f_origin = pd.merge(f_origin, pp_sum, left_on='orig_port_cd', right_on='Port', how='left')

# Keep only the origin port codes and their corresponding daily capacities
f_origin = f_origin[['orig_port_cd', 'Daily Capacity ']]

# Convert the list of unique origin port codes to a Python list
f_origin_list = f_origin['orig_port_cd'].tolist()

# Output the list of unique origin port codes
f_origin_list


['PORT02',
 'PORT03',
 'PORT04',
 'PORT05',
 'PORT06',
 'PORT07',
 'PORT08',
 'PORT09',
 'PORT10',
 'PORT11']

In [30]:
# Merge the unique destination ports DataFrame ('f_dest') with the 'pp_sum' DataFrame
# This adds the corresponding 'Daily Capacity' for each destination port
f_dest = pd.merge(f_dest, pp_sum, left_on='dest_port_cd', right_on='Port', how='left')

# Keep only the destination port codes and their corresponding daily capacities
f_dest = f_dest[['dest_port_cd', 'Daily Capacity ']]

# Convert the list of unique destination port codes to a Python list
f_dest_list = f_dest['dest_port_cd'].tolist()

# Output the list of unique destination port codes
f_dest_list


['PORT09']

In [31]:
# Create a dictionary mapping each origin port code to its corresponding daily capacity
# Use 'zip' to pair 'orig_port_cd' with 'Daily Capacity' from 'f_origin'
# Convert the 'Daily Capacity' column to float for numerical operations
origin_cap = dict(zip(f_origin['orig_port_cd'], f_origin['Daily Capacity '].astype('float')))

# Create a similar dictionary mapping each destination port code to its corresponding daily capacity
# Use 'zip' to pair 'dest_port_cd' with 'Daily Capacity' from 'f_dest'
# Convert the 'Daily Capacity' column to float for numerical operations
dest_cap = dict(zip(f_dest['dest_port_cd'], f_dest['Daily Capacity '].astype('float')))

# Output the dictionary for origin ports and their capacities
origin_cap


{'PORT02': 1453.0,
 'PORT03': 138.0,
 'PORT04': 2076.0,
 'PORT05': 554.0,
 'PORT06': 434.0,
 'PORT07': 549.0,
 'PORT08': 11.0,
 'PORT09': 457.0,
 'PORT10': 8.0,
 'PORT11': 111.0}

In [32]:
dest_cap


{'PORT09': 457.0}

In [33]:
# Define the linear programming problem with the objective to minimize transportation costs
problem_statement = LpProblem("Port Distribution Problem", LpMinimize)

# Create decision variables for transportation routes between origin and destination ports
# Each variable represents the quantity of goods transported from an origin to a destination
variables = LpVariable.dicts("route", (f_origin_list, f_dest_list), 0, None, LpInteger)

# Generate all possible transportation routes as (origin, destination) pairs
routes = [(o, d) for o in f_origin_list for d in f_dest_list]

# Create a dictionary of transportation costs for each route
costs = makeDict([f_origin_list, f_dest_list], cost_pulp_2, 0)

# Define the objective function to minimize the total transportation cost
problem_statement += (
    lpSum([variables[o][d] * costs[o][d] for (o, d) in routes]),  # Sum of cost * quantity for all routes
    "Sum_of_Transporting_Costs"
)

# Add constraints to ensure the total weight leaving each origin port does not exceed its capacity
for o in f_origin_list:
    problem_statement += (
        lpSum([variables[o][d] for d in f_dest_list]) <= origin_cap[o],  # Total weight leaving origin <= capacity
        "Sum_of_Weight_out_of_Port_%s" % o  # Unique constraint name for each origin
    )

# Add constraints to ensure the total weight arriving at each destination port meets its demand
for d in f_dest_list:
    problem_statement += (
        lpSum([variables[o][d] for o in f_origin_list]) >= dest_cap[d],  # Total weight arriving at destination >= demand
        "Sum_of_Weight_in_Port_%s" % d  # Unique constraint name for each destination
    )

# Solve the linear programming problem
problem_statement.solve()

# Print the solution, including the values of decision variables (quantities transported)
for a in problem_statement.variables():
    print(f'{a} : {a.varValue}')


/Users/surya/anaconda3/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/surya/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j7/t594hs655zzf0dkp2sm786kr0000gn/T/eca9bdd9dd6c48f6a4d167acabd491a8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/j7/t594hs655zzf0dkp2sm786kr0000gn/T/eca9bdd9dd6c48f6a4d167acabd491a8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 67 RHS
At line 79 BOUNDS
At line 90 ENDATA
Problem MODEL has 11 rows, 10 columns and 20 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7312 - 0.00 seconds
Cgl0004I processed model has 1 rows, 10 columns (10 integer (0 of which binary)) and 10 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 7312 found by greedy cover after 0 iterations and 0 nodes (0.02 seconds)
Cbc0001I Search com

# Interpretation:

The solution indicates that only the route from PORT07 to PORT09 is used to minimize transportation costs while satisfying all constraints.

Ports PORT02, PORT03, and others do not contribute to transportation to PORT09, as their inclusion would increase the total cost.

Non-Zero Allocation:
route_PORT07_PORT09: 457.0 units are transported from PORT07 to PORT09.

Zero Allocation:
All other routes (e.g., route_PORT02_PORT09, route_PORT03_PORT09, etc.) have a transportation quantity of 0.0, indicating they are not part of the optimal solution.
